<a href="https://colab.research.google.com/github/shivendra1-cyber/AmazonWebScrapping2/blob/main/AmazonWebScrapping2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PROBLEM STATEMENT
The Ecommerce URL is given below and search for table fans and need to extract the product information according to mentioned headers on the excel as follows:
https://www.amazon.in/
Headers to be required on Excel:


a)
Product Name

b)
Product Code or SKU ID

c)
Price

d)
Dimensions

e)
Weight

f)
Manufacturer Name

g)
Warranty

h)
Features

i)
Color

j)
Other Relevant Information




#SOLUTION

Step 1: Remove Old Chrome & ChromeDriver
Run this first to clean up old installations:

In [50]:
!apt-get remove -y google-chrome-stable
!rm -rf /usr/bin/google-chrome /usr/bin/chromedriver
!apt-get autoremove -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libvulkan1 mesa-vulkan-drivers
Use 'apt autoremove' to remove them.
The following packages will be REMOVED:
  google-chrome-stable
0 upgraded, 0 newly installed, 1 to remove and 37 not upgraded.
After this operation, 374 MB disk space will be freed.
(Reading database ... 126821 files and directories currently installed.)
Removing google-chrome-stable (134.0.6998.165-1) ...
update-alternatives: using /usr/bin/chromium-browser to provide /usr/bin/x-www-browser (x-www-browser) in auto mode
update-alternatives: using /usr/bin/chromium-browser to provide /usr/bin/gnome-www-browser (gnome-www-browser) in auto mode
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The fol

Step 2: Install Latest Google Chrome

In [51]:
!apt-get update
!apt-get install -y wget unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || true
!apt-get -f install -y
!rm google-chrome-stable_current_amd64.deb
!google-chrome --version


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,684 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,784 kB]
Fetched 11.5 MB in 3s (3,410 kB/s)
Reading package lists

Step 3: Install Matching ChromeDriver for Chrome 134
Run this command to download the correct ChromeDriver version:

In [52]:
!wget -O chromedriver.zip https://storage.googleapis.com/chrome-for-testing-public/134.0.6998.165/linux64/chromedriver-linux64.zip
!unzip chromedriver.zip
!mv chromedriver-linux64/chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver
!chromedriver --version


--2025-03-27 14:02:41--  https://storage.googleapis.com/chrome-for-testing-public/134.0.6998.165/linux64/chromedriver-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.12.27, 142.250.65.123, 172.217.164.27, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.12.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9494938 (9.1M) [application/zip]
Saving to: ‘chromedriver.zip’

chromedriver.zip    100%[===================>]   9.05M  --.-KB/s    in 0.04s   

2025-03-27 14:02:41 (225 MB/s) - ‘chromedriver.zip’ saved [9494938/9494938]

Archive:  chromedriver.zip
  inflating: chromedriver-linux64/LICENSE.chromedriver  
  inflating: chromedriver-linux64/THIRD_PARTY_NOTICES.chromedriver  
  inflating: chromedriver-linux64/chromedriver  
ChromeDriver 134.0.6998.165 (fd886e2cb29dd984c13deec032832dee68d8abe3-refs/branch-heads/6998@{#2122})


Step 4: Test ChromeDriver

In [53]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.binary_location = "/usr/bin/google-chrome"

# Correct ChromeDriver path
service = Service("/usr/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=chrome_options)

# Test if it works
driver.get("https://www.google.com")
print("✅ ChromeDriver is working fine!")
driver.quit()


✅ ChromeDriver is working fine!


In [80]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import random

# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Define ChromeDriver path
service = Service("/usr/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=chrome_options)

# Open Amazon & search for "Table Fans"
url = "https://www.amazon.in/s?k=table+fans"
driver.get(url)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@data-component-type='s-search-result']")))

# Extract product details
products = driver.find_elements(By.XPATH, "//div[@data-component-type='s-search-result']")

data = []
for index, product in enumerate(products):
    try:
        name = product.find_element(By.XPATH, ".//span[contains(@class, 'a-size-medium')]").text.strip()
    except:
        name = "N/A"

    try:
        asin = product.get_attribute("data-asin")
    except:
        asin = "N/A"

    try:
        price = product.find_element(By.XPATH, ".//span[@class='a-price-whole']").text.strip()
    except:
        price = "N/A"

    try:
        product_link = product.find_element(By.XPATH, ".//a[@class='a-link-normal s-no-outline']").get_attribute("href")
    except:
        product_link = "N/A"

    # Open product details page
    dimensions, weight, manufacturer, warranty, features, color = ["N/A"] * 6
    if product_link != "N/A":
        driver.execute_script("window.open(arguments[0]);", product_link)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(random.uniform(2, 4))  # Randomized delay

        def get_detail(text):
            try:
                return WebDriverWait(driver, 5).until(EC.presence_of_element_located(
                    (By.XPATH, f"//th[contains(text(), '{text}')]/following-sibling::td")
                )).text.strip()
            except:
                return "N/A"

        dimensions = get_detail("Product Dimensions")
        weight = get_detail("Item Weight")
        manufacturer = get_detail("Manufacturer")
        warranty = get_detail("Warranty")
        features = get_detail("Features")
        color = get_detail("Colour")

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # Back to main tab

    data.append([name, asin, price, dimensions, weight, manufacturer, warranty, features, color, product_link])

    if index >= 9:
        break  # Limit to avoid blocking

# Save data to Excel
df = pd.DataFrame(data, columns=["Product Name", "Product Code (ASIN)", "Price", "Dimensions", "Weight",
                                 "Manufacturer Name", "Warranty", "Features", "Color", "Product Link"])
df.to_excel("Amazon_Table_Fans.xlsx", index=False)

print("✅ Data Scraping Complete! File saved as 'Amazon_Table_Fans.xlsx'.")

driver.quit()


✅ Data Scraping Complete! File saved as 'Amazon_Table_Fans.xlsx'.


In [82]:
from google.colab import files
files.download("amazon_products.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>